## Dato il Trump Twitter Archive acquisire due language model (Bi-grammi e Tri-grammi) e usarli per produrre Tweet

In [13]:
#Import and constants
import random

import nltk
import csv

In [14]:
#Utilities
def read_csv(filename):
    dataset=[]
    with open(filename, 'r',encoding='utf-8') as file:
        # Create a CSV reader object
        reader = csv.reader(file)
        for row in reader:
            if(not row[1][0]=="@"):
                sentence_bi = "[" + row[1].replace("&amp","") + "]"
                sentence_tri = "[[" + row[1].replace("&amp","") + "]]"
                #Per ogni tweet aggiungo una versione con un token di start e un token di fine e una versione con due token
                #di start e due token di fine da usare per i bi-grammi e i tri-grammi
                dataset.append((sentence_bi,sentence_tri))
    return dataset

# Language model a Bi-grammi

In [15]:
#Data una sequenza di token rappresentate tutti i tweet del dataset
#Estraggo tutti i bigrammi sotto forma di un dictonary contenente dictonary
#Salvando le frequenze di ogni bigramma
#Ad esempio per il bigramma (i,eat) sarà salvato in bigram_counts[i] un dictionary tale che bigram_counts[i][eat]=1
def count_bigrams(tokens):
    bigram_counts = {}
    for i in range(len(tokens) - 1):
        current_token = tokens[i]
        next_token = tokens[i + 1]
        if current_token not in bigram_counts:
            bigram_counts[current_token] = {}
        if next_token not in bigram_counts[current_token]:
            bigram_counts[current_token][next_token] = 0
        bigram_counts[current_token][next_token] += 1
    return bigram_counts


def learn_bi_gram_model(dataset):
    #Creo un array di token composto dalla sequenza di token di tutti i tweet del dataset
    tokens = [nltk.word_tokenize(sentence) for sentence in [elem[0] for elem in dataset]]
    tokens = [token for sublist in tokens for token in sublist]
    #Conto i bi-grammi
    bigram_counts = count_bigrams(tokens)

    #Calcolo il numero di parole del vocabolario
    vocab_size = len(set(tokens))

    #Inserisco le probabilità in biTwitter_probabilities con Laplace Smoothing
    biTwitter_probabilities = {}
    #Per ogni primo token dei bigrammi
    for first_token in bigram_counts:
        sum=0
        #Calcolo il conteggio di tutti i bigrammi che iniziano con il primo token
        for second_token in bigram_counts[first_token]:
            sum+=bigram_counts[first_token][second_token]
        #Per ogni secondo token nel bigramma
        for second_token in bigram_counts[first_token]:
            #Creo il bigramma come tupla (Utile dopo per la generazione)
            bi_gram=(first_token,second_token)
            #Calcolo la probabilità del singolo bigramma
            biTwitter_probabilities[bi_gram] = (bigram_counts[first_token][second_token] + 1) / (sum + vocab_size)

    return biTwitter_probabilities

def generate_text_bigram(prob):
    #Uso come token iniziale per la generazione il token di start
    current_word = "["
    generated_text = [current_word]
    #Finchè non genero un token di fine o supero la lunghezza massima
    while current_word != "]" and len(generated_text) < 50:
        #Calcolo i possibili bigrammi da generare selezionandoli sulla base del token attuale
        possible_next_bigrams = [bigram for bigram in prob if bigram[0] == current_word]
        #Estraggo le probabilità dei prossimi bigrammi
        probabilities = [prob[(bigram)] for bigram in possible_next_bigrams]
        #Faccio una scelta casuale lavorando sulle probabilità dei bigrammi
        next_word = random.choices(possible_next_bigrams, probabilities)[0][1]
        generated_text.append(next_word)
        current_word = next_word
        #Una prima implementazione non faceva una scelta casuale ma prendeva deterministicamente sempre il primo.
        #Questa implementazione però ricorreva spesso in loop infiniti ed è stata sotituita con una più casuale
    return " ".join(generated_text[1:-1])

# Main e uso del modello a bi-grammi

In [16]:
 # Lettura CSV
file_path = 'trump_twitter_archive/tweets.csv'
df = read_csv(file_path)

bi_prob=learn_bi_gram_model(df)
print("BIGRAM GENERATED SENTENCES")
print(generate_text_bigram(bi_prob))
print(generate_text_bigram(bi_prob))
print(generate_text_bigram(bi_prob))

BIGRAM GENERATED SENTENCES
The ruling @ UnionLeader . Cruz is a protected 2nd Amendment biggest ) goes to the winners ; losers out of which sadly there are last on Wednesday January 17th rather be weak ; losers refuse to admire about Doral bedbugs but the nickname Mini Mike was biggest
Another attack in Madison Square Garden during my best wishes to watch Celebrity @ AGSchneiderman ’ s phony last on being on Tariffs ; prosper be our rapidly rebuilding Military Vets ( cont ) picking winners for his dumb as National Magazine and losers of the haters and
I have it ...


# Language model a Tri-grammi

In [17]:
#Data una sequenza di token rappresentate tutti i tweet del dataset
#Estraggo tutti i trigrammi sotto forma di un dictonary con chiave un bigramma contenente dictonary
#Salvando le frequenze di ogni trigramma
#Ad esempio per il trigramma (i,eat,ramen) sarà salvato in trigram_counts[(i,eat)] un dictionary tale che bigram_counts[(i,eat)][ramen]=1
def count_trigrams(tokens):
    trigram_counts = {}
    for i in range(len(tokens) - 2):
        current_token = tokens[i]
        next_token = tokens[i + 1]
        next_next_token = tokens[i + 2]
        if (current_token, next_token) not in trigram_counts:
            trigram_counts[(current_token, next_token)] = {}
        if next_next_token not in trigram_counts[(current_token, next_token)]:
            trigram_counts[(current_token, next_token)][next_next_token] = 0
        trigram_counts[(current_token, next_token)][next_next_token] += 1
    return trigram_counts

def learn_tri_gram_model(dataset):
    #Creo un array di token composto dalla sequenza di token di tutti i tweet del dataset
    tokens = [nltk.word_tokenize(sentence) for sentence in [elem[1] for elem in dataset]]
    tokens = [token for sublist in tokens for token in sublist]
    #Conto i tri-grammi
    trigram_counts = count_trigrams(tokens)
    #Calcolo il numero di parole del vocabolario
    vocab_size = len(set(tokens))

    #Inserisco le probabilità in triTwitter_probabilities con Laplace Smoothing
    triTwitter_probabilities = {}
    #Per ogni primo bigramma iniziale dei trigrammi
    for first_bigram in trigram_counts:
        sum = 0
        #Calcolo il conteggio di tutti i tri-grammi che iniziano con il primo bi-gramma
        for third_token in trigram_counts[first_bigram]:
            sum += trigram_counts[first_bigram][third_token]
         #Per ogni terzo token nel bigramma
        for third_token in trigram_counts[first_bigram]:
            #Creo il tri-gramma
            tri_gram = (first_bigram[0],first_bigram[1], third_token)
            #Calcolo la probabilità
            triTwitter_probabilities[tri_gram] = (trigram_counts[first_bigram][third_token] + 1) / (sum + vocab_size)
    return triTwitter_probabilities

def generate_text_trigram(prob):
    #Uso i token di start come tri-gramma iniziale per la generazione
    current_trigram = ("[", "[", "[")
    generated_text = list(current_trigram)
    #Finchè non genero i token di fine o supero la soglia massima
    while current_trigram[-2:] != ("]", "]") and len(generated_text) < 50:
        #Se ho già generato almeno 6 caratteri genero il prossimo trigramma in modo deterministico
        if (len(generated_text) > 6):
            #Calcolo il trigramma con la probabilità maggiore dato il bigramma attuale
            next_trigram = max(prob, key=lambda tri_gram: tri_gram[:2] == current_trigram[-2:])
            generated_text.append(next_trigram[-1])
            current_trigram = next_trigram
        else:
            #Genero il prossimo bigramma in modo più casuale in modo da non generare sempre lo stesso tweet
            #Calcolo tutti i possibili prossimi trigrammi
            possible_next_trigrams = [trigram for trigram in prob if
                                      trigram[:2] == current_trigram[-2:]]
            #Mappo le loro probabilità
            probabilities = [prob[trigram] for trigram in possible_next_trigrams]
            #Faccio una selezione casuale basata sulle probabilità
            next_trigram = random.choices(possible_next_trigrams, probabilities)[0]
            generated_text.append(next_trigram[-1])
            current_trigram = next_trigram

    return " ".join(generated_text[3:-2])

# Uso del modello a tri-grammi

In [18]:
tri_prob=learn_tri_gram_model(df)
print("\nTRIGRAM GENERATED SENTENCES")
print(generate_text_trigram(tri_prob))
print(generate_text_trigram(tri_prob))
print(generate_text_trigram(tri_prob))


TRIGRAM GENERATED SENTENCES
Wow I ’ m at 2200000 followers but I did build a world class loser Tim O ’ Brien who I haven ’ t know how to win and their so-called Lincoln Project ” goes into their own pockets . With what I ’ ve
Steyer is a LOSER who has money but can ’ t know how to win and their so-called Lincoln Project ” goes into their own pockets . With what I ’ ve done on Judges Taxes Regulations Healthcare the Military Vets ( Choice ! )
LOSER ! https : //t.co/p5imhMJqS1
